# PDF inlezen

In [32]:
from langdetect import detect
import fitz # PyMuPDF
import nltk

In [33]:
pdf = fitz.open("../verslag\output\CluyseDylanBP.pdf")

De inhoudstafel ophalen en uitlezen. De uitvoer is een 2D-array met waarden: hoofdstuknummer, titel, de pagina waarop het hoofdstuk / de sectie start.

In [34]:
pdf.get_toc()

[[1, 'Lĳst van figuren', 7],
 [1, 'Inleiding', 9],
 [2, 'Probleemstelling', 10],
 [2, 'Onderzoeksvraag', 10],
 [2, 'Onderzoeksdoelstelling', 11],
 [2, 'Opzet van deze bachelorproef', 11],
 [1, 'Stand van zaken', 13],
 [2, 'Onderzoeken rond dyslexie', 13],
 [3, 'Centraal zicht op dyslexie', 13],
 [3, 'Fonologische dyslexie', 14],
 [3, 'Centraal zicht op de doelgroep', 14],
 [3, 'Diagnosecriteria', 15],
 [3, 'Moeilijkheden bij dyslexie', 16],
 [3, 'Aandachtspunten bij ondersteuning', 18],
 [3,
  'Bewezen voordelen van tekstvereenvoudiging bij scholieren met dyslexie',
  18],
 [2, 'Wetenschappelijke artikelen', 19],
 [3, 'Wetenschappelijke geletterdheid in Vlaanderen', 20],
 [3, 'Trends rond wetenschappelijke artikelen', 21],
 [3, 'Formaat', 23],
 [3, 'Woordenschat en vakjargon', 24],
 [3, 'Aanpak voor het lezen van wetenschappelijke artikelen', 24],
 [3, 'Conclusie', 25],
 [2, 'Tekstvereenvoudiging', 25],
 [3, 'Manuele tekstvereenvoudiging', 26],
 [3, 'Natural Language Processing', 27],


Embeddings van de pdf ophalen.

In [35]:
pdf.embfile_names()

[]

Boolean die aanwijst of een bestand de pdf-extensie heeft.

In [36]:
pdf.is_pdf

True

Boolean dat aanwijst of een bestand het gepaste pdf-formaat heeft.

In [37]:
pdf.is_form_pdf

False

In [38]:
pdf.is_reflowable

False

Boolean dat aanwijst of een wachtwoord nodig is om de pdf te lezen.

In [39]:
pdf.needs_pass

False

Metadata van de pdf ophalen.

In [40]:
pdf.metadata

{'format': 'PDF 1.5',
 'title': 'Scholieren met dyslexie van de derde graad middelbaar onderwijs ondersteunen bij het lezen van wetenschappelijke papers via tekstsimplificatie',
 'author': 'Dylan Cluyse',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'xdvipdfmx (20200315)',
 'creationDate': "D:20230308142407-00'00'",
 'modDate': '',
 'trapped': '',
 'encryption': None}

In [41]:
pdf.metadata.get('title')

'Scholieren met dyslexie van de derde graad middelbaar onderwijs ondersteunen bij het lezen van wetenschappelijke papers via tekstsimplificatie'

In [42]:
pdf.chapter_count

1

Functie dat de pagina's per hoofdstuk/sectie opvraagt. Het bereik is ``[start van hoofdstuk, start van volgend hoofdstuk - 1].`` De start wordt op 999 geplaatst, als er van uit wordt gegaan dat er geen pdf wordt geüpload met meer dan 999 pagina's.

In [43]:
def get_pages_per_title(chosen_title):
    start = 999
    end = pdf.page_count - 1
    for title in pdf.get_toc():
        if title[1] == chosen_title:
            start = title[2]

        if title[2] >= start and title[1] != chosen_title:
            end = title[2]
            break

    return start, end

In [44]:
get_pages_per_title('Tekstvereenvoudiging')

(25, 26)

In [45]:
get_pages_per_title('Trends rond wetenschappelijke artikelen')

(21, 23)

Inhoud ophalen per bereik ``[Start, einde]``. Alle tekst tussen de tekst bovenaan de startpagina en onderaan de laatste pagina wordt opgenomen.

In [46]:
def get_content_per_range(start, end):
    text = ""
    for i in range(start, end + 1):
        page = pdf.load_page(i)
        text += page.get_text()
    return text

## Voorbeeld:
1. Start- en eindpagina van een hoofdstuk opvragen.
2. Alle tekstinhoud van dit hoofdstuk opvragen.
3. Onnodige tekens en regeleindes verwijderen.
4. Eerste 500 tokens uitprinten.

In [47]:
title = 'Trends rond wetenschappelijke artikelen'
start, end = get_pages_per_title(title)
text = get_content_per_range(start, end)

In [69]:
import re
def text_cleaning(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\d+\.\d+\.\d+\.', '', text) # 2.2.2 Stand van Zaken verwijderen.
    text = text.strip('.')
    return text

In [49]:
text = text_cleaning(text)
print(text[:500])

14 2. Stand van zaken kers (Ball, 2017; Jones e.a., 2019; Plavén-Sigray e.a., 2017). Plavén-Sigray e.a. (2017) onderzoekt de verschillende trends waarom wetenschap- pelijke artikelen alsmaar moeilijker te lezen worden. De relatie tussen de leesbaar- heid van een abstract werd vergeleken met het jaar waarin het wetenschappelijk artikel werd gepubliceerd. De Flesch-Reading-Ease of FRE score werd gebruikt om de leesgraad van een wetenschappelijk artikel te beoordelen. Om te bevesti- gen dat de rela


Taal achterhalen

In [50]:
language = detect(text)
language

'nl'

# Counting tokens

There are still too many tokens. We need to resort to an in-between method: extractive summarization.

In [51]:
from nltk import FreqDist
fdist = FreqDist(text)
fdist.N()

5431

## Conversion to sentences

In [52]:
text = nltk.sent_tokenize(text)
text

['14 2.',
 'Stand van zaken kers (Ball, 2017; Jones e.a., 2019; Plavén-Sigray e.a., 2017).',
 'Plavén-Sigray e.a.',
 '(2017) onderzoekt de verschillende trends waarom wetenschap- pelijke artikelen alsmaar moeilijker te lezen worden.',
 'De relatie tussen de leesbaar- heid van een abstract werd vergeleken met het jaar waarin het wetenschappelijk artikel werd gepubliceerd.',
 'De Flesch-Reading-Ease of FRE score werd gebruikt om de leesgraad van een wetenschappelijk artikel te beoordelen.',
 'Om te bevesti- gen dat de relatie tussen de complexiteit van een abstract overeenstemt met die van de volledige tekstinhoud, werden er vergelijkingen gemaakt met zes verschil- lende wetenschappelijke journalen.',
 'De overeenkomst tussen de leesgraad van het abstract en de overige tekstinhoud in een wetenschappelijk artikel werd eerder be- vestigd door Dronberger en Kowitz (1975).',
 'Dat onderzoek benadrukte dat een ab- stract complexer werd geschreven, vergeleken met de rest van een wetenschappe- 

## Extractive summarization

Note: Both do not currently work in this set-up. TODO

In [53]:
# https://github.com/wietsedv/bertje

from summarizer import Summarizer
from transformers import AutoTokenizer, AutoModel, TFAutoModel
from transformers import BertTokenizer, TFBertLMHeadModel

tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased", revision="v1")
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")

# https://github.com/iPieter/RobBERT

from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")

Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing Robert

In [54]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(40000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [55]:
tokenizer

PreTrainedTokenizer(name_or_path='pdelobelle/robbert-v2-dutch-base', vocab_size=40000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

### Chunk creation

In [56]:
text = " ".join(text)

In [57]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Extractive summarization with BERT

In [58]:
summarizer = Summarizer()

result = summarizer(
    #algorithm=...,
    body=' '.join(sentences),
    max_length=460,
    min_length=60,
    num_sentences=4,
    #ratio=...,
    #return_as_list=...,
    #use_first=...,
)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
result

'2017) onderzoekt de verschillende trends waarom wetenschap- pelijke artikelen alsmaar moeilijker te lezen worden. Het onderzoek schat dat nu een kwart van alle wetenschappelijke artikelen gebruik maken van Engels op het niveau van een masterstudent, ofwel een FRE onder nul. Wetenschappe- lijke artikels worden enkel toegankelijk tot mensen die wetenschappelijk geletterd zijn of een leesgraad daarboven hebben. Sommige wetenschappers zoeken direct onbekende woorden op of raadplegen extra informatiebronnen, terwijl an- dere wetenschappers hoofdstukken overslaan.'

In [60]:
fdist = FreqDist(' '.join(result))
fdist.N()

1127

# Abstractive summarization + text simplification

In [173]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import configparser, os

COMPLETIONS_MODEL = "text-davinci-003"
config = configparser.ConfigParser()
config.read('../config.ini')
openai.api_key = config['openai']['api_key']

Example of abstractive summarization (no language explicitly stated).

In [181]:
amount_words_sentence = 10
amount_sentences = 1

prompt = f"""
Vereenvoudig deze tekst met deze parameters:
Zin is max {amount_words_sentence} woorden lang
Max {amount_sentences} aantal zinnen.
Schrijf dit met zo een eenvoudig mogelijke woordenschat.
context: 
{text}
"""

fdist = FreqDist(prompt)
fdist.N()

5590

In [175]:
openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Wetenschappelijke artikelen zijn moeilijk te lezen. Onderzoek toont aan dat een kwart van de artikelen Engels gebruikt op het niveau van een masterstudent. Dit beperkt de toegankelijkheid en reproduceerbaarheid. Onderwijs moet stilstaan bij de woordenschat en grammatica. Wetenschappers gebruiken verschillende tactieken om artikelen te begrijpen.'

Example of explaining a complex term. 

In [182]:
chosen_word = "reproduceerbaarheid"

prompt = f"""
Wat is {chosen_word} in de context van {title}
"""

fdist = FreqDist(prompt)
fdist.N()

86

In [183]:
openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Reproduceerbaarheid is de mate waarin de resultaten van een wetenschappelijk artikel kunnen worden herhaald door andere onderzoekers. Het is een belangrijk principe in de wetenschap, omdat het de betrouwbaarheid van de resultaten vergroot. Reproduceerbaarheid is een van de trends die momenteel worden gezien in wetenschappelijke artikelen, waarbij onderzoekers worden aangemoedigd om hun methoden en resultaten zo transparant mogelijk te maken, zodat anderen hun werk kunnen herhalen.'

## Determining the hard-to-read words with Flesch-Kincaid

In [184]:
import textstat

Sentence-level

In [188]:
s = 'Wetenschappelijke artikelen zijn moeilijk te lezen. Onderzoek toont aan dat een kwart van de artikelen Engels gebruikt op het niveau van een masterstudent. Dit beperkt de toegankelijkheid en reproduceerbaarheid. Onderwijs moet stilstaan bij de woordenschat en grammatica. Wetenschappers gebruiken verschillende tactieken om artikelen te begrijpen.'

# Calculate readability scores
fk_grade = textstat.flesch_kincaid_grade(s)
fk_reading_ease = textstat.flesch_reading_ease(s)

# Format results into a sentence
result = "The Flesch-Kincaid Grade level of the text is {}, and the Flesch Reading Ease score is {}.".format(fk_grade, fk_reading_ease)

# Print result
print(result)

The Flesch-Kincaid Grade level of the text is 11.5, and the Flesch Reading Ease score is 28.5.


## Extractive-abstractive summarization for a long document

In [ ]:
def ask_gpt(extracted_text):
    amount_words_sentence = 10
    amount_sentences = 3

    prompt = f"""
    Vereenvoudig deze tekst met deze parameters:
    Zin is max {amount_words_sentence} woorden lang
    Max {amount_sentences} aantal zinnen.
    Schrijf dit met zo een eenvoudig mogelijke woordenschat.
    context: 
    {extracted_text}
    """

    return openai.Completion.create(
        prompt=prompt,
        temperature=0,
        max_tokens=300,
        model=COMPLETIONS_MODEL)["choices"][0]["text"].strip(" \n")

In [72]:
full_summarized_doc = []

for title in pdf.get_toc()[3:]:
    start, end = get_pages_per_title(title[1])
    text = get_content_per_range(start, end)
    cleaned_text = text_cleaning(text)
    
    
    result = summarizer(
        #algorithm=...,
        body=text,
        max_length=460,
        min_length=60,
        num_sentences=4,
        #ratio=...,
        #return_as_list=...,
        #use_first=...,
    )
    
    full_summarized_doc.append(ask_gpt(result))

In [73]:
full_summarized_doc[:500]

'Onderzoeksvraag\nDe volgende onderzoeksvraag is opgesteld: ”Hoe kan een wetenschappelijke arti-\nkel automatisch vereenvoudigd worden, gericht op de unieke noden van scholie-\nren met dyslexie in de derde graad middelbaar onderwijs?”. • Welke specifieke noden hebben scholieren met dyslexie van de derde graad\nmiddelbaar onderwijs bij het begrijpen van complexere teksten? • Wat zijn de specifieke kenmerken van wetenschappelijke artikelen? Inleiding\nIn Hoofdstuk 3 wordt de methodologie toegelicht en w'